In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install flask-ngrok pyngrok joblib
!ngrok config add-authtoken 2vonfmiSthOr5GkHfJHYhpaNwis_2nwh9v2zQGRU9cSGUZ264


Mounted at /content/drive
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
!pip install flask-ngrok pyngrok

import os
import pandas as pd
from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)

UPLOAD_FOLDER = '/tmp'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
df = None
index = 0

HTML_PAGE = """
<!DOCTYPE html>
<html>
<head>
    <title> AI Based Real Time Blackout Prediction </title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        body {
            font-family: 'Comic Sans MS', cursive, sans-serif;
            background: linear-gradient(135deg, #fce4ec, #e0f7fa);
            text-align: center;
            padding: 50px;
            color: #555;
        }
        h2 {
            font-size: 28px;
            color: #d63384;
            margin-bottom: 20px;
        }
        form {
            margin-bottom: 30px;
        }
        input[type="file"], button {
            padding: 10px 20px;
            border: none;
            border-radius: 12px;
            font-size: 16px;
            margin: 5px;
        }
        input[type="file"] {
            background-color: #fff;
            border: 2px solid #f8bbd0;
            color: #d63384;
        }
        button {
            background-color: #f8bbd0;
            color: white;
            cursor: pointer;
        }
        button:hover {
            background-color: #f48fb1;
        }
        canvas {
            background: white;
            border-radius: 15px;
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <h2> AI Based Real Time Blackout Prediction</h2>
    <form id="form" enctype="multipart/form-data">
        <input type="file" name="file" required />
        <button type="submit">Start</button>
    </form>
    <canvas id="chart" width="800" height="400"></canvas>

    <script>
        let chart;
        const labels = [];
        const datasets = {
            'Current': [],
            'Voltage': [],
            'Power': [],
            'Blackout': []
        };

        const ctx = document.getElementById('chart').getContext('2d');
        chart = new Chart(ctx, {
            type: 'line',
            data: {
                labels: labels,
                datasets: Object.keys(datasets).map((key, idx) => ({
                    label: key,
                    borderColor: ['#f48fb1', '#81d4fa', '#ce93d8', '#a1887f'][idx],
                    backgroundColor: 'transparent',
                    data: datasets[key],
                    borderWidth: 2,
                    tension: 0.3,
                    pointRadius: 2,
                }))
            },
            options: {
                responsive: true,
                animation: {
                    duration: 500
                },
                scales: {
                    x: {
                        title: {
                            display: true,
                            text: 'Index',
                            color: '#888'
                        }
                    },
                    y: {
                        title: {
                            display: true,
                            text: 'Sensor Value',
                            color: '#888'
                        }
                    }
                },
                plugins: {
                    legend: {
                        labels: {
                            color: '#555',
                            font: {
                                size: 14,
                                family: 'Comic Sans MS'
                            }
                        }
                    }
                }
            }
        });

        document.getElementById('form').addEventListener('submit', function(e) {
            e.preventDefault();
            const formData = new FormData(this);
            fetch('/upload', {
                method: 'POST',
                body: formData
            }).then(() => {
                setInterval(() => {
                    fetch('/next')
                        .then(res => res.json())
                        .then(data => {
                            if (data.done) return;
                            labels.push(data.index);
                            Object.keys(datasets).forEach(key => {
                                datasets[key].push(data[key]);
                            });
                            chart.update();
                        });
                }, 500);
            });
        });
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_PAGE)

@app.route('/upload', methods=['POST'])
def upload():
    global df, index
    file = request.files['file']
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(filepath)
    df = pd.read_csv(filepath)
    if not all(col in df.columns for col in ['Current', 'Voltage', 'Power', 'Blackout']):
        return 'Missing required columns', 400
    index = 0
    return 'Uploaded', 200

@app.route('/next')
def next_row():
    global index, df
    if df is None or index >= len(df):
        return jsonify({'done': True})
    row = df.iloc[index]
    data = {
        'index': index,
        'Current': row['Current'],
        'Voltage': row['Voltage'],
        'Power': row['Power'],
        'Blackout': row['Blackout'],
        'done': False
    }
    index += 1
    return jsonify(data)

# 🚀 LAUNCH
public_url = ngrok.connect(5000)
print("🚀 OPEN THIS LINK:", public_url)
app.run()


🚀 OPEN THIS LINK: NgrokTunnel: "https://3bc6-104-155-210-208.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3bc6-104-155-210-208.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 16:28:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 16:28:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 16:28:48] "POST /upload HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 16:28:49] "POST /upload HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 16:28:49] "POST /upload HTTP/1.1" 400 -
ERROR:__main__:Exception on /upload [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File